In [ ]:
import datalabframework
from pyspark.sql.functions import col
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import Window
from datalabframework.spark.utils import unidecode
import pandas as pd
import os
from datalabframework.spark import dataframe
import numpy as np
os.environ['PYSPARK_PYTHON'] = '/opt/conda/bin/python'
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import matplotlib.pyplot as plt
from scipy.signal import periodogram
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import scipy.stats as scs
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt, ExponentialSmoothing
import math
from statsmodels.tsa.arima_model import ARIMA
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder

In [ ]:
#load fact
dlf = datalabframework.project.load()
engine = dlf.engine()
spark = engine.context()
fact_transaction = engine.load('fact_table').select('sku_id', 'sku_name', 'transaction_date', 'quantity', 'doc_type', 'unit_price')

## Load data from fact

In [ ]:
#extract quantity 
product_quantity_date = fact_transaction.where(F.expr('doc_type == "PTX"') | F.expr('doc_type == "HDF"'))\
                .where(F.expr('unit_price != 0'))\
                .groupby('sku_id', 'sku_name', 'transaction_date')\
                .agg(F.sum('quantity').alias('daily_total'), F.avg('unit_price').alias('daily_price'))\
                .orderBy('transaction_date')
#extract_all_quantity
product_quantity = fact_transaction.where(F.expr('doc_type == "PTX"') | F.expr('doc_type == "HDF"'))\
                                   .groupby('sku_id', 'sku_name')\
                                   .agg(F.sum('quantity').alias('total_quantity'), F.avg('unit_price').alias('avg_price'))\
                                   .orderBy(F.desc('total_quantity'))

In [ ]:
product_quantity.show(20, truncate = False)

In [ ]:
total_product = product_quantity_date.where('sku_id != "1206838"')\
                                     .where('sku_id != "1207652"')\
                                     .groupby('transaction_date').agg(F.sum(F.col('daily_total')).alias('daily_total_product'))
total_product.show(5)

### Choose an arbitrary product

In [ ]:
one_ex_product = product_quantity_date.where('sku_id == 1204431').toPandas()
one_ex_product['transaction_date'] = one_ex_product['transaction_date'].astype(np.datetime64)
one_ex_product['daily_total'] = one_ex_product['daily_total'].astype(np.int64)
one_ex_product['daily_price'] = one_ex_product['daily_price'].astype(np.float)/1000000
start_date = one_ex_product['transaction_date'].min()
end_date = one_ex_product['transaction_date'].max()
one_ex_product = one_ex_product.set_index('transaction_date')
full_date = pd.DataFrame({'date_ts':pd.date_range(start_date, end_date)}).set_index('date_ts')
full_quantity_product = one_ex_product.join(full_date, how = 'outer').fillna(0)

### Total product

In [ ]:
pd_total_product = total_product.toPandas()
pd_total_product['daily_total_product'] = pd_total_product['daily_total_product'].astype(np.int64)
start_date = pd_total_product['transaction_date'].min()
end_date = pd_total_product['transaction_date'].max()
pd_total_product = pd_total_product.set_index('transaction_date')
full_date = pd.DataFrame({'date_ts':pd.date_range(start_date, end_date)}).set_index('date_ts')
full_quantity_total_product = pd_total_product.join(full_date, how = 'right').fillna(0)

In [ ]:
full_quantity_total_product['week'] = full_quantity_total_product.index.to_series().dt.week
full_quantity_total_product['year'] = full_quantity_total_product.index.to_series().dt.year
full_quantity_total_product['month'] = full_quantity_total_product.index.to_series().dt.month
full_quantity_total_product['dow'] = full_quantity_total_product.index.to_series().dt.dayofweek
full_quantity_total_product['dom'] = full_quantity_total_product.index.to_series().dt.day

In [ ]:
agg_week= full_quantity_total_product.groupby('dow').agg({'daily_total_product': 'mean'})
agg_week.columns = ['agg_total_week']
agg_week.loc[4].values

In [ ]:
def fix_value_tet_holiday(row):
    if row['daily_total_product'] == 0:
        return agg_week.loc[row.loc['dow']].values[0]
    return row['daily_total_product']

In [ ]:
full_quantity_total_product['daily_total_product'] = full_quantity_total_product.apply(fix_value_tet_holiday, axis = 1)

In [ ]:
full_quantity_total_product_dummies = pd.get_dummies(full_quantity_total_product, prefix = ['dow', 'moy', 'dom'], columns = ['dow', 'month', 'dom'])

In [ ]:
holiday_info = pd.read_csv('special_day.csv')
holiday_info.columns = ['date', 'special_day_code', 'special_day_name']
holiday_info = holiday_info.set_index('date')
full_quantity_total_product_dummies_holi = full_quantity_total_product_dummies.join(holiday_info)
full_quantity_total_product_dummies_holi.head(5)

In [ ]:
full_quantity_total_product_dummies_holi = pd.get_dummies(full_quantity_total_product_dummies_holi, prefix = 'holiday', \
                                                         columns = ['special_day_code'], drop_first = True)

In [ ]:
full_quantity_total_product_dummies_holi.head(5)

In [ ]:
full_quantity_total_product_dummies.shape

### Test satationary

In [ ]:
def test_stationarity(timeseries):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [ ]:
test_stationarity(full_quantity_total_product['daily_total_product'])

### Analysis

In [ ]:
plt.figure(figsize = (15, 6))
plt.plot(full_quantity_total_product_dummies.index, full_quantity_total_product_dummies['daily_total_product'])

In [ ]:
plt.plot(full_quantity_total_product['daily_total_product'].rolling(120).mean())

In [ ]:
#visualize mean over time
x = full_quantity_total_product['daily_total_product'].values
y = [np.mean(x[:t]) for t in range(len(x))]
plt.plot(y[1:])

In [ ]:
#visualize variance over time
x = full_quantity_total_product['daily_total_product'].values
y = [np.std(x[:t]) for t in range(len(x))]
plt.plot(y[1:])

In [ ]:
#ACF
smt.graphics.plot_acf(full_quantity_total_product['daily_total_product'], lags = 90);

In [ ]:
#PACF
smt.graphics.plot_pacf(full_quantity_total_product['daily_total_product'], lags = 90);

In [ ]:
#Try diff 1
x_diff1 = full_quantity_total_product['daily_total_product'].diff(1)
plt.figure(figsize = (15, 6))
plt.plot(x_diff1)
for i in range(290):
    for k in range(3000):
        priont(what am I doing now)
for k in range(200):
    for k in range(30)

In [ ]:
test_stationarity(x_diff1[1:])
test_stationarity(x_diff[2:])
test_stationarity(x_diff[:3])
for i in range(2000):
    for j in range(2300):
        fo

In [ ]:
smt.graphics.plot_acf(x_diff1[1:], lags = 90);

In [ ]:
smt.graphics.plot_pacf(x_diff1[1:], lags = 90);

In [ ]:
#try diff 7
x_diff7 = full_quantity_total_product['daily_total_product'].diff(7).dropna()
plt.figure(figsize = (15, 6))
plt.plot(x_diff7)

In [ ]:
test_stationarity(x_diff7)

In [ ]:
smt.graphics.plot_acf(x_diff7, lags = 90);

In [ ]:
smt.graphics.plot_pacf(x_diff7, lags = 90);
smt.graphics.plot_acf(x_diff7, lags = 20)

In [ ]:
#x_diff1_diff7
x_diff1_diff7 = full_quantity_total_product['daily_total_product'].diff(1).diff(7).dropna()
plt.figure(figsize = (15, 6))
plt.plot(x_diff1_diff7)

In [ ]:
smt.graphics.plot_acf(x_diff1_diff7, lags = 90);

In [ ]:
smt.graphics.plot_pacf(x_diff1_diff7, lags = 90);

In [ ]:
res = sm.tsa.seasonal_decompose(x = full_quantity_total_product['daily_total_product'], freq = 30, model = 'additive', extrapolate_trend=30)
res.plot();

In [ ]:
test_stationarity(res.trend.diff(1).diff(1).dropna())

In [ ]:
smt.graphics.plot_acf(res.trend.dropna(), lags = 40);

In [ ]:
smt.graphics.plot_acf(res.trend.dropna().diff(1).dropna(), lags = 100);

In [ ]:
smt.graphics.plot_pacf(res.trend.dropna().diff(1).dropna(), lags = 100);

In [ ]:
res1 = sm.tsa.seasonal_decompose(res.trend.dropna(), freq = 365, model = 'additive')
res1.plot();

In [ ]:
plt.plot(np.log(res1.trend.dropna()))

In [ ]:
plt.plot(res1.trend)

### Correlation between x and lags of x

In [ ]:
x = full_quantity_total_product['daily_total_product']
plt.scatter(x.shift(1), x)

In [ ]:
plt.scatter(x.shift(7), x)

In [ ]:
plt.scatter(x.shift(30), x)

In [ ]:
def split_train_test(test_number, data):
    train_number = data.shape[0] - test_number
    train_series = data.iloc[:train_number]
    test_series = data.iloc[train_number:]
    return train_series, test_series

In [ ]:
#test AR(1)
x = full_quantity_total_product['daily_total_product'].values
x_lag1 = full_quantity_total_product['daily_total_product'].values[1:]
plt.scatter(x[:-1], x_lag1)
x_arr = np.array(x[:-1]).reshape(-1, 1)
y_arr = np.array(x_lag1).reshape(-1, 1)
linear_model = LinearRegression().fit(x_arr, y_arr)
print('Score: ',linear_model.score(x_arr, y_arr))
print('Coef:', linear_model.coef_)
plt.plot(x_arr, linear_model.predict(x_arr), c = 'r')
plt.xlabel('lag1')
plt.ylabel('quantity')
plt.title('Scatter plot of quantity vs lag1')

In [ ]:
from statsmodels.tsa.ar_model import AR
ar_model = AR(x).fit(1)

In [ ]:
model = ARIMA(x, (1, 0, 0)).fit(method = 'mle', trend = 'c')
print(model.summary())

In [ ]:
predicted_value = linear_model.predict(x_arr).ravel()
residual = predicted_value - x_arr.ravel()

plt.scatter(predicted_value, residual, c = 'r')

### All method definition

In [ ]:
def find_summary_arima(train_series, p, d, q):
    try:
        model = ARIMA(train_series, (p, d, q), freq = 'D')
        res = model.fit(method = 'mle', trend = 'nc')
        aic_value = res.aic
    except:
        return None, None
    return aic_value, res

In [ ]:
def find_best_arima_order(train_series):
    rng = range(4)
    best_aic = np.inf
    best_order = None
    best_mdl = None
    for p in rng:
        for d in rng:
            for q in rng:
                aic_value, mdl = find_summary_arima(train_series, p, d, q)
                if (aic_value != None):
                    if (aic_value < best_aic):
                        best_aic = aic_value
                        best_order = (p, d, q)
                        best_mdl = mdl
    return best_order, best_mdl

In [ ]:
train_day_series, test_day_series = split_train_test(30,full_quantity_total_product_dummies)

In [ ]:
decompose_parts = sm.tsa.seasonal_decompose(train_series[target_col], freq = 7, model = 'additive')
decompose_parts.plot();

In [ ]:
plt.boxplot(decompose_parts.resid.dropna()['daily_total_product'].values);

In [ ]:
x = decompose_parts.resid.dropna()['daily_total_product'].values
np.quantile(x, 0.25), np.quantile(x, 0.75)

In [ ]:
#trend
remove_seasonal = decompose_parts.observed - decompose_parts.seasonal
plt.plot(remove_seasonal);

In [ ]:
#ARIMA for train
#see the acf_plot
best_order, best_mdl = find_best_arima_order(remove_seasonal)

In [ ]:
#let's predict
n_train = train_series.shape[0]
n_test = test_series.shape[0]
trend_residual_predict, t1, t2 = best_mdl.forecast(n_test)
trend_residual_predict = pd.Series(trend_residual_predict, index = test_series.index)

In [ ]:
seasonal_test = [decompose_parts.seasonal.values[(i + n_train)%7][0] for i in range(n_test)]
predict_test = trend_residual_predict + seasonal_test

In [ ]:
def arima_seasonal(train_series, test_series, predictor_cols, target_col):
    decompose_parts = sm.tsa.seasonal_decompose(train_series[target_col], freq = 7, model = 'additive')
    remove_seasonal = decompose_parts.observed - decompose_parts.seasonal
    best_order, best_mdl = find_best_arima_order(remove_seasonal)
    n_train = train_series.shape[0]
    n_test = test_series.shape[0]
    trend_residual_predict, t1, t2 = best_mdl.forecast(n_test)
    trend_residual_predict = pd.Series(trend_residual_predict, index = test_series.index)
    seasonal_for_test = [decompose_parts.seasonal.values[(i + n_train)%7][0] for i in range(n_test)]
    predict_for_test = trend_residual_predict + seasonal_for_test
    return predict_for_test, None, None

In [ ]:
def arima_seasonal_month(train_series, test_series, predictor_cols, target_col):
    decompose_parts = sm.tsa.seasonal_decompose(train_series[target_col], freq = 30, model = 'additive')
    remove_seasonal = decompose_parts.observed - decompose_parts.seasonal
    best_order, best_mdl = find_best_arima_order(remove_seasonal)
    n_train = train_series.shape[0]
    n_test = test_series.shape[0]
    trend_residual_predict, t1, t2 = best_mdl.forecast(n_test)
    trend_residual_predict = pd.Series(trend_residual_predict, index = test_series.index)
    seasonal_for_test = [decompose_parts.seasonal.values[(i + n_train)%7][0] for i in range(n_test)]
    predict_for_test = trend_residual_predict + seasonal_for_test
    return predict_for_test, None, None

In [ ]:
def average_method(train_series, test_series, predictor_cols, target_col):
    avg = np.mean(train_series[target_col])
    preds = np.full(test_series.shape[0], avg)
    return preds, None, None

In [ ]:
def naive_method(train_series, test_series, predictor_cols, target_col):
    last_value = train_series.values[-1]
    preds = np.full(train_series.shape[0], last_value)
    return preds, None, None

In [ ]:
def drift_method(train_series, test_series, predictor_cols, target_col):
    X = np.arange(train_series.shape[0]).reshape(-1,1)
    y = np.array(train_series[target_col])
    x_ = np.arange(train_series.shape[0], train_series.shape[0] + test_series.shape[0]).reshape(-1, 1)
    linear_model = LinearRegression().fit(X, y)
    preds = linear_model.predict(x_)
    return preds, linear_model, linear_model.predict(X) - y

In [ ]:
def AR1(train_series, test_series, predictor_cols, target_col):
    model = ARIMA(train_series[target_col].values, (1, 0, 0)).fit(method = 'mle', trend = 'c')
    print('param:',model.params)
    preds,t1,t2 = model.forecast(test_series.shape[0])
    return preds, model, model.resid

In [ ]:
def naive_season_method(train_series, test_series, predictor_cols, target_col):
    X = train_series[predicted_cols]
    X['index'] = np.arange(train_series.shape[0]).reshape(-1,1)
    X = np.array(X)
    y = np.array(train_series[target_col])
    x_ = test_series[predicted_cols]
    x_['index'] = np.arange(train_series.shape[0], train_series.shape[0] + test_series.shape[0]).reshape(-1, 1)
    x_ = np.array(x_)
    linear_model = LinearRegression().fit(X, y)
    preds = linear_model.predict(x_)
    return preds, linear_model, linear_model.predict(X) - y

In [ ]:
def upgrade_season_method(train_series, test_series, predictor_cols, target_col):
    X = train_series[predicted_cols]
    X['index'] = np.arange(train_series.shape[0]).reshape(-1,1)
    X = np.array(X)
    y = np.array(train_series[target_col])
    x_ = test_series[predicted_cols]
    x_['index'] = np.arange(train_series.shape[0], train_series.shape[0] + test_series.shape[0]).reshape(-1, 1)
    x_ = np.array(x_)
    linear_model = LinearRegression().fit(X, y)
    resid = linear_model.predict(X) - y
    resid_series = pd.Series(resid[:,0], train_series.index)
    best_order, best_mdl = find_best_arima_order(resid_series)
    seasonal_for_test = linear_model.predict(x_)[:,0]
    trend_residual_predict, t1, t2 = best_mdl.forecast(n_test)
    trend_residual_predict = pd.Series(trend_residual_predict, index = test_series.index)
    predict_for_test = trend_residual_predict + seasonal_for_test
    return predict_for_test, None, None

In [ ]:
def forecasting_process(train_series, test_series, func_model, type_model, predictor_cols, target_col):
    total_mse = 0
    total_mad = 0
    total_mape = 0
    n = 0
    result_columns = ['period', 'demand', 'forecast', 'error', 'abs_error','MSE', 'MAD', 'percent_error', 'MAPE']
    result = pd.DataFrame([], columns = result_columns)
    selected_cols = list(set(predictor_cols).union(set(target_col)))
    preds, model, resid = func_model(train_series[selected_cols], test_series[selected_cols], predictor_cols, target_col)
    for index, row in test_series.iterrows():
        pred = preds[n]
        error = pred - row[target_col].values[0]
        abs_error = math.fabs(error)
        percent_error = abs_error/row[target_col].values[0]
        n = n + 1
        total_mse += error ** 2
        total_mad += abs_error
        total_mape += percent_error
        mse = total_mse/n
        mad = total_mad/n
        mape = total_mape/n
        v = [index, row[target_col].values[0], pred, error, abs_error, mse, mad, percent_error, mape]
        result = result.append([dict(zip(result_columns, v))])
    result.index = np.arange(test_series.shape[0])
    f = plt.figure(figsize = (15, 6))
    plt.plot(train_series.index, train_series[target_col].values, label = 'train')
    plt.plot(test_series.index,test_series[target_col].values, label = 'test')
    plt.plot(test_series.index, result['forecast'], color = 'k', label = 'forecast')
    plt.xlabel('day')
    plt.ylabel('quantity')
    plt.title(type_model)
    plt.legend();
    return result, model, resid

### Test ARIMA Seasonal

In [ ]:
arimaseasonal_result, model, resid = forecasting_process(train_day_series, test_day_series, arima_seasonal, 'arima_seasonal', [], ['daily_total_product'])

### Test avg method

In [ ]:
avg_result, model, resid = forecasting_process(train_day_series, test_day_series, average_method, 'avarage', [], ['daily_total_product'])

### Test Naive method

In [ ]:
naive_result, model, resid = forecasting_process(train_day_series, test_day_series, naive_method, 'Naive', [], ['daily_total_product'])

### Test drift method

In [ ]:
drift_result, model,resid = forecasting_process(train_day_series, test_day_series, drift_method, 'drift', [], ['daily_total_product'])

In [ ]:
plt.plot(resid, alpha = 0.7, label = 'varaiance = {:.3f}'.format(np.std(resid)))
plt.hlines(0, xmin = 0, xmax = len(resid), color = 'r')
plt.legend();

In [ ]:
smt.graphics.plot_acf(resid, lags = 400);

### Test AR(1)

In [ ]:
ar1_result, model, resid = forecasting_process(train_day_series, test_day_series, AR1, 'AR1', [], ['daily_total_product'])


In [ ]:
plt.plot(resid, alpha = 0.7, label = 'varaiance = {:.3f}'.format(np.std(resid)))
plt.hlines(0, xmin = 0, xmax = len(resid), color = 'r')
plt.legend();

In [ ]:
smt.graphics.plot_acf(resid, lags = 400);

### Test Seasonal Naive

In [ ]:
predicted_cols = list(set(train_day_series.columns) - set(['daily_total_product', 'week', 'year']))
target_col = ['daily_total_product']
seasonal_naive_result, model, resid = forecasting_process(train_day_series, test_day_series, naive_season_method, 'naive', predicted_cols, target_col)

In [ ]:
plt.plot(resid, alpha = 0.7, label = 'varaiance = {:.3f}'.format(np.std(resid)))
plt.hlines(0, xmin = 0, xmax = len(resid), color = 'r')
plt.legend()

In [ ]:
smt.graphics.plot_acf(resid, lags = 100);

### Test upgrade season method

In [ ]:
predicted_cols = list(set(train_day_series.columns) - set(['daily_total_product', 'week', 'year']))
target_col = ['daily_total_product']
upgrade_seasonal_result, model, resid = forecasting_process(train_day_series, test_day_series, upgrade_season_method, 'upgrade_season', predicted_cols, target_col)

### Compare all based methods

In [ ]:
plt.figure(figsize = (10, 8))
all_result_name = ['arimaseasonal_result', 'avg', 'naive', 'drift', 'ar1', 'season_naive', 'upgrade_seasonal']
all_result_method = [arimaseasonal_result, avg_result, naive_result, drift_result, ar1_result, seasonal_naive_result, upgrade_seasonal_result]
for result_name, result_method in zip(all_result_name, all_result_method):
    plt.plot(result_method['MAPE'], label = result_name)
plt.xlabel('test_size')
plt.ylabel('MAPE')
plt.title('MAPE for all methods')
plt.legend();

In [ ]:
plt.plot(full_quantity_product[['daily_total']]);

In [ ]:
res = sm.tsa.seasonal_decompose(full_quantity_product[['daily_total']], freq = 30, model = 'additive')
res.plot();

In [ ]:
res1 = sm.tsa.seasonal_decompose(res.trend.dropna(), freq = 356, model = 'additive')
res1.plot();

In [ ]:
res1 = sm.tsa.seasonal_decompose(res.trend.dropna(), freq = 356, model = 'multiplicative')
res1.plot();

In [ ]:
res_week = sm.tsa.seasonal_decompose(week_data['weekly_total'], freq = 53, model = 'additive')
fig = res_week.plot()

In [ ]:
np.sum(res_week.resid**2)/len(res_week.resid)

In [ ]:
smt.graphics.plot_acf(week_data['weekly_total'], alpha=0.5);
smt.graphics.plot_pacf(week_data['weekly_total'], alpha = 0.5);

In [ ]:
def test_stationarity(timeseries):
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

In [ ]:
test_stationarity(week_data['weekly_total'])

In [ ]:
week_data_series = week_data[['weekly_total']]

In [ ]:
test_number = 8
train_number = week_data_series.shape[0] - test_number
train_series = week_data_series.loc[:train_number - 1]
test_series = week_data_series.loc[train_number:]

In [ ]:
def simple_exp_smoothing(train_series, test_series, type_model):
    total_mse = 0
    total_mad = 0
    total_mape = 0
    n = 0
    result_columns = ['period', 'demand', 'forecast', 'error', 'abs_error','MSE', 'MAD', 'percent_error', 'MAPE']
    result = pd.DataFrame([], columns = result_columns)
    if type_model == 'SimpleExpSmoothing':
        model = SimpleExpSmoothing(train_series.values[:, 0])
        fit = model.fit()
    elif type_model == 'Holt':
        model = Holt(train_series.values[:, 0])
        fit = model.fit(smoothing_level = 0.3, smoothing_slope = 0.05)
    preds = fit.forecast(test_series.shape[0])
    for index, row in test_series.iterrows():
        pred = preds[n]
        error = pred - row.values[0]
        abs_error = math.fabs(error)
        percent_error = abs_error/row.values[0]
        n = n + 1
        total_mse += error ** 2
        total_mad += abs_error
        total_mape += percent_error
        mse = total_mse/n
        mad = total_mad/n
        mape = total_mape/n
        v = [index, row.values[0], pred, error, abs_error, mse, mad, percent_error, mape]
        result = result.append([dict(zip(result_columns, v))])
        train_series = train_series.append(pd.DataFrame([(row.values[0],)], index = [index], columns = train_series.columns))
    f = plt.figure(figsize = (15, 6))
    plt.plot(train_series.index, train_series.values[:, 0])
    plt.plot(test_series.index,test_series.values[:, 0])
    plt.plot(test_series.index, result['forecast'], color = 'k')
    plt.title(type_model)
    plt.legend();
    return result

In [ ]:
result = simple_exp_smoothing(train_series, test_series, 'SimpleExpSmoothing')
result

In [ ]:
result = simple_exp_smoothing(train_series, test_series, 'Holt')
result

In [ ]:
def find_summary_arima(train_series, p, d, q):
    try:
        model = ARIMA(train_series, (p, d, q))
        res = model.fit(method = 'mle', trend = 'nc')
        aic_value = res.aic
    except:
        return None, None
    return aic_value, res

In [ ]:
def find_best_arima_order(train_series):
    rng = range(4)
    best_aic = np.inf
    best_order = None
    best_mdl = None
    for p in rng:
        for d in rng:
            for q in rng:
                aic_value, mdl = find_summary_arima(train_series, p, d, q)
                if (aic_value != None):
                    if (aic_value < best_aic):
                        best_aic = aic_value
                        best_order = (p, d, q)
                        best_mdl = mdl
    return best_order, best_mdl

In [ ]:
def arima_process(train_series, test_series):
    total_mse = 0
    total_mad = 0
    total_mape = 0
    n = 0
    result_columns = ['period', 'demand', 'forecast', 'error', 'abs_error','MSE', 'MAD', 'percent_error', 'MAPE', 'best_order']
    result = pd.DataFrame([], columns = result_columns)
    best_order, best_mdl = find_best_arima_order(train_series)
    preds, t1, t2= best_mdl.forecast(test_series.shape[0])
    for index, row in test_series.iterrows():
        pred = preds[n]
        error = pred - row.values[0]
        abs_error = math.fabs(error)
        percent_error = abs_error/row.values[0]
        n = n + 1
        total_mse += error ** 2
        total_mad += abs_error
        total_mape += percent_error
        mse = total_mse/n
        mad = total_mad/n
        mape = total_mape/n
        v = [index, row.values[0], pred, error, abs_error, mse, mad, percent_error, mape, best_order]
        result = result.append([dict(zip(result_columns, v))])
        train_series = train_series.append(pd.DataFrame([(row.values[0],)], index = [index], columns = train_series.columns))
    plt.figure(figsize = (15, 6))
    plt.plot(train_series.index, train_series.values[:, 0])
    plt.plot(test_series.index,test_series.values[:, 0])
    plt.plot(test_series.index, result['forecast'], color = 'k')
    plt.title('ARIMA')
    plt.legend();
    return result

In [ ]:
result = arima_process(train_series, test_series)
result

In [ ]:
def seasonalExponentialSmoothing(train_series, test_series):
    total_mse = 0
    total_mad = 0
    total_mape = 0
    n = 0
    result_columns = ['period', 'demand', 'forecast', 'error', 'abs_error','MSE', 'MAD', 'percent_error', 'MAPE']
    result = pd.DataFrame([], columns = result_columns)
    result = pd.DataFrame([], columns = result_columns)
    model = ExponentialSmoothing(train_series.values[:, 0], trend = 'add', seasonal = 'add', seasonal_periods = 53)
    fit = model.fit()
    preds = fit.forecast(test_series.shape[0])
    for index, row in test_series.iterrows():
        pred = preds[n]
        error = pred - row.values[0]
        abs_error = math.fabs(error)
        percent_error = abs_error/row.values[0]
        n = n + 1
        total_mse += error ** 2
        total_mad += abs_error
        total_mape += percent_error
        mse = total_mse/n
        mad = total_mad/n
        mape = total_mape/n
        v = [index, row.values[0], pred, error, abs_error, mse, mad, percent_error, mape]
        result = result.append([dict(zip(result_columns, v))])
        train_series = train_series.append(pd.DataFrame([(row.values[0],)], index = [index], columns = train_series.columns))
    f = plt.figure(figsize = (15, 6))
    plt.plot(train_series.index, train_series.values[:, 0])
    plt.plot(test_series.index,test_series.values[:, 0])
    plt.plot(test_series.index, result['forecast'], color = 'k')
    plt.title('Seasonal exponential smoothing')
    plt.legend();
    return result

In [ ]:
result = seasonalExponentialSmoothing(train_series, test_series)
result

In [ ]:
train_series.iloc[0].values[0]

In [ ]:
def simpleBootstrap(len_block, train_series, test_series):
    #moving block bootstrap
    #create data points
    size_train = train_series.shape[0]
    n_points = train_series.shape[0] - len_block + 1
    X = np.zeros((n_points, len_block - 1))
    y = np.zeros(n_points)
    for i in range(n_points):
        for j in range(len_block - 1):
            X[i,j] = train_series.iloc[i + j].values[0]
        y[i] = train_series.iloc[i + len_block - 1].values[0]
    liner_model = LinearRegression().fit(X, y)
    total_mse = 0
    total_mad = 0
    total_mape = 0
    n = 0
    result_columns = ['period', 'demand', 'forecast', 'error', 'abs_error','MSE', 'MAD', 'percent_error', 'MAPE']
    result = pd.DataFrame([], columns = result_columns)
    values = np.zeros(len_block - 1)
    for i in range(len_block - 1):
        values[i] = train_series.iloc[train_series - len_block + i + 2]
    for index, row in test_series.iterrows():
        pred = liner_model.predict(values)
        error = pred - row.values[0]
        abs_error = math.fabs(error)
        percent_error = abs_error/row.values[0]
        n = n + 1
        total_mse += error ** 2
        total_mad += abs_error
        total_mape += percent_error
        mse = total_mse/n
        mad = total_mad/n
        mape = total_mape/n
        v = [index, row.values[0], pred, error, abs_error, mse, mad, percent_error, mape]
        result = result.append([dict(zip(result_columns, v))])
        train_series = train_series.append(pd.DataFrame([(row.values[0],)], index = [index], columns = train_series.columns)